In [1]:
%%capture
!pip install morfessor

In [2]:
%%capture
!pip install indic-nlp-library

In [3]:
%%capture
!pip install googletrans==3.1.0a0

In [4]:
%%capture
!pip install translingual

In [5]:
import numpy as np
from tqdm.notebook import tqdm
from collections import OrderedDict
import pickle
from googletrans import Translator, constants
import time
import json
import os
import glob
from random import uniform

# Importing raw data

Data: https://nlp.stanford.edu/data/glove.6B.zip

In [6]:
from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/Colab Notebooks/Projects/PIE/"

with open(project_path + "Embeddings/english_embeddings.pickle", 'rb') as f:
    english_embeddings_dict = pickle.load(f)

Mounted at /content/drive


In [7]:
INDIC_NLP_RESOURCES=r"/content/drive/MyDrive/Colab Notebooks/Projects/PIE/indic_nlp_resources-master"

from indicnlp.transliterate.unicode_transliterate import ItransTransliterator

from indicnlp import common, loader
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()


The composition of the dictionary is as follows: corresponding to each word, there is an array of length matching the dimensions of the GloVe embeddings chosen.

# Translation

In [8]:
src_lang_dict = english_embeddings_dict
dest_language_code = 'hi'
dest_language = 'hindi'
dest_language_dict = OrderedDict()

In [9]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]

In [11]:

from txtai.pipeline import Translation

# Create translation model
translate = Translation()

KeyError: ignored

In [ ]:
translation = translate("This is a test translation into hindi", "hi")
translation

In [ ]:
count = 0
#translator = Translator()
embeddings_path = project_path + 'Translated_Embeddings/' + dest_language
list_of_files = glob.glob(embeddings_path + "/*")
try:
    latest_count = int(max(list_of_files, key=os.path.getctime).replace(embeddings_path + '/' + dest_language + '_language_dict_' , "").replace('.json', ""))
except ValueError:
    latest_count = 0

src_lang_dict_items_list = list(src_lang_dict.items())

for k, v in tqdm(src_lang_dict_items_list[latest_count:]):
    #translator = Translator()
    translated_word = translate(k, "hi")
    romanised_word = ItransTransliterator.to_itrans(translated_word, dest_language_code)
    dest_language_dict[translated_word] = v.tolist()
    count += 1
    #time.sleep(uniform(3, 4))
    if count % 30 == 0:
        print(translated_word, romanised_word)
        #time.sleep(uniform(10, 20))
    if count % 100 == 0:
        filename = embeddings_path + '/' + dest_language + '_language_dict_' + str(latest_count+1000) + '.json'
        with open(filename, 'w') as f:
            json.dump(dest_language_dict, f)
            #break
        dest_language_dict = OrderedDict()


In [ ]:
count = 1
embeddings_path = project_path + 'Translated_Embeddings/' + dest_language
list_of_files = glob.glob(embeddings_path + "/*")
try:
    latest_count = int(max(list_of_files, key=os.path.getctime).replace(embeddings_path + '/' + dest_language + '_language_dict_' , "").replace('.json', ""))
except ValueError:
    latest_count = 0

src_lang_dict_items_list = list(src_lang_dict.items())

c = 0
list1 = []
for k, v in tqdm(src_lang_dict_items_list[latest_count:]):
    if count % 10 != 0:
        print('no', count)
        count += 1
        list1 += [k, v]
    else:
        print('yes')
        translator = Translator()
        list2betranslated = [kvpair[0] for kvpair in list1]
        translated_list = [translation.text for translation in translator.translate(list2betranslated, dest=dest_language_code)]
        #romanised_word = ItransTransliterator.to_itrans(translated_word, dest_language_code)
        list1 = [[translated_list[list1.index(kvpair)], kvpair[1]] for kvpair in list1]
        dest_language_dict.update({kvpair[0] : kvpair[1] for kvpair in list1})
        #dest_language_dict[translated_word] = v.tolist()
        count += 1
        time.sleep(uniform(3, 4))
        if True: #if count % 30 == 0:
            print(translated_list)
            time.sleep(uniform(10, 20))
        if count % 100 == 0:
            filename = embeddings_path + '/' + dest_language + '_language_dict_' + str(latest_count+1000) + '.json'
            with open(filename, 'w') as f:
                json.dump(dest_language_dict, f)
                #break
            dest_language_dict = OrderedDict()



In [ ]:
count = 0
#translator = Translator()
embeddings_path = project_path + 'Translated_Embeddings/' + dest_language
list_of_files = glob.glob(embeddings_path + "/*")
try:
    latest_count = int(max(list_of_files, key=os.path.getctime).replace(embeddings_path + '/' + dest_language + '_language_dict_' , "").replace('.json', ""))
except ValueError:
    latest_count = 0

src_lang_dict_items_list = list(src_lang_dict.items())

for k, v in tqdm(src_lang_dict_items_list[latest_count:]):
    translator = Translator()
    translated_word = translator.translate(k, dest=dest_language_code).text
    romanised_word = ItransTransliterator.to_itrans(translated_word, dest_language_code)
    dest_language_dict[translated_word] = v.tolist()
    count += 1
    time.sleep(uniform(3, 4))
    if count % 10 == 0:
        print(translated_word, romanised_word)
        time.sleep(uniform(10, 20))
    if count % 100 == 0:
        filename = embeddings_path + '/' + dest_language + '_language_dict_' + str(latest_count+1000) + '.json'
        with open(filename, 'w') as f:
            json.dump(dest_language_dict, f)
            #break
        dest_language_dict = OrderedDict()


In [ ]:
list(dest_language_dict.keys()).index('seine')

Concatenate translated embeddings

In [ ]:
"""
master_filename = project_path + 'Translated_Embeddings/' + dest_language + '/master/' + dest_language + '_language_dict' + '.json'
for count in [a for a in range(0, 40000) if a % 10 == 0]:
    filename = project_path + 'Translated_Embeddings/' + dest_language + '/' + dest_language + '_language_dict_' + str(count) + '.json'
    with open(filename, 'r') as j:
        contents = json.loads(j.read())
        with open(master_filename, 'w') as fp:
            fp.append('a')

In [ ]:
#da = json.loads(filename, object_pairs_hook=OrderedDict)

In [ ]:
'''
print(constants.LANGUAGES)
'''

In [ ]:
'''
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

corpus_size = file_len(file_path)
corpus_size
'''

In [ ]:
exit()

In [ ]:
'''
def create_trans_dict_and_file(src_lang_file_path, dest_language_code, dest_language_path, dest_language_dict_pre):
  with open(src_lang_file_path, 'r', encoding="utf-8") as f_src:
    for line in tqdm(src_lang_file_path):
      count = 0
      word = line.split()[0]
      vector = np.asarray(line[1:], 'float32')
      translated_word = translator.translate(word, dest=dest_language_code).text
      translated_line = line.replace(word, translated_word)
      with open(dest_language_path, 'a') as f_dest:
        f_dest.write(translated_line)
      dest_language_dict_pre[word] = vector
      count += 1
      if count == 6:
        break
  f_src.close()
  return dest_language_dict
'''